In [11]:
# Importar bibliotecas utilizadas
import pandas as pd

import numpy as np




In [63]:
# Set of predefined words NLTK (Natural Language Toolkit)
import nltk 

from sklearn.feature_extraction.text import CountVectorizer

# Regular expression
import re

from nltk.tokenize import word_tokenize

from nltk.corpus import stopwords 
#from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
set(stopwords.words('english'))

print(stopwords.words('english'))



['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jcaet\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
# Coleta dados
catalogoVulnerabilidades = pd.read_csv('known_exploited_vulnerabilities.csv', header=0, sep=',')

# Criar coluna booleana state com 0 para Unknown e 1 para Known
catalogoVulnerabilidades['codigoKnownRansomwareCampaignUse'] = np.where(catalogoVulnerabilidades['knownRansomwareCampaignUse'] == 'Known', 1, 0 )

#Reordenando as colunas
catalogoVulnerabilidades = catalogoVulnerabilidades[['cveID', 
                                                     'vendorProject',
                                                     'product',
                                                     'vulnerabilityName',
                                                     'dateAdded',
                                                     'shortDescription',
                                                     'requiredAction',
                                                     'dueDate',
                                                     'knownRansomwareCampaignUse',
                                                     'codigoKnownRansomwareCampaignUse',
                                                     'notes']]    



In [14]:
# Verificar tipos de colunas
catalogoVulnerabilidades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1082 entries, 0 to 1081
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   cveID                             1082 non-null   object
 1   vendorProject                     1082 non-null   object
 2   product                           1082 non-null   object
 3   vulnerabilityName                 1082 non-null   object
 4   dateAdded                         1082 non-null   object
 5   shortDescription                  1082 non-null   object
 6   requiredAction                    1082 non-null   object
 7   dueDate                           1082 non-null   object
 8   knownRansomwareCampaignUse        1082 non-null   object
 9   codigoKnownRansomwareCampaignUse  1082 non-null   int32 
 10  notes                             315 non-null    object
dtypes: int32(1), object(10)
memory usage: 88.9+ KB


In [15]:
# Excluir a coluna notes
catalogoVulnerabilidades.drop(columns=['notes'], inplace = True)

In [24]:
# Verificar alguns dados
catalogoVulnerabilidades.head()

,cveID,vendorProject,product,vulnerabilityName,dateAdded,shortDescription,requiredAction,dueDate,knownRansomwareCampaignUse,codigoKnownRansomwareCampaignUse
0,CVE-2021-27104,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,Known,True
1,CVE-2021-27102,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,Known,True
2,CVE-2021-27101,Accellion,FTA,Accellion FTA SQL Injection Vulnerability,2021-11-03,Accellion FTA contains a SQL injection vulnera...,Apply updates per vendor instructions.,2021-11-17,Known,True
3,CVE-2021-27103,Accellion,FTA,Accellion FTA Server-Side Request Forgery (SSR...,2021-11-03,Accellion FTA contains a server-side request f...,Apply updates per vendor instructions.,2021-11-17,Known,True
4,CVE-2021-21017,Adobe,Acrobat and Reader,Adobe Acrobat and Reader Heap-based Buffer Ove...,2021-11-03,Acrobat Acrobat and Reader contain a heap-base...,Apply updates per vendor instructions.,2021-11-17,Unknown,False


In [17]:
# Transformar tipos de dados
catalogoVulnerabilidades.dateAdded = pd.to_datetime(catalogoVulnerabilidades.dateAdded)
catalogoVulnerabilidades.dueDate = pd.to_datetime(catalogoVulnerabilidades.dueDate)
##Tranforma a coluna codigoKnownRansomwareCampaignUse para boolean
catalogoVulnerabilidades['codigoKnownRansomwareCampaignUse'] = catalogoVulnerabilidades['codigoKnownRansomwareCampaignUse'].astype('bool')
##Transforma colunas em string
catalogoVulnerabilidades['shortDescription'] = catalogoVulnerabilidades['shortDescription'].astype('string')

In [18]:
# Verificar tipos de colunas após transformação
catalogoVulnerabilidades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1082 entries, 0 to 1081
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   cveID                             1082 non-null   object        
 1   vendorProject                     1082 non-null   object        
 2   product                           1082 non-null   object        
 3   vulnerabilityName                 1082 non-null   object        
 4   dateAdded                         1082 non-null   datetime64[ns]
 5   shortDescription                  1082 non-null   string        
 6   requiredAction                    1082 non-null   object        
 7   dueDate                           1082 non-null   datetime64[ns]
 8   knownRansomwareCampaignUse        1082 non-null   object        
 9   codigoKnownRansomwareCampaignUse  1082 non-null   bool          
dtypes: bool(1), datetime64[ns](2), object(6), string

In [64]:
# Função para limpar os textos das colunas do dataframe
def limpa_coluna(corpus):
    return re.sub(r'[^a-zA-Z0-9\s]', '', corpus)

In [68]:
# Limpa a coluna 
pd_catalogoVulnerabilidades = pd.DataFrame(catalogoVulnerabilidades)
pd_catalogoVulnerabilidades['shortDescription'] = pd_catalogoVulnerabilidades['shortDescription'].apply(lambda x: limpa_coluna(str(x).lower()))

pd_catalogoVulnerabilidades.shortDescription

0       accellion fta contains an os command injection...
1       accellion fta contains an os command injection...
2       accellion fta contains a sql injection vulnera...
3       accellion fta contains a serverside request fo...
4       acrobat acrobat and reader contain a heapbased...
                              ...                        
1077    microsoft windows internet shortcut files cont...
1078    microsoft windows smartscreen contains a secur...
1079    cisco adaptive security appliance asa and fire...
1080    microsoft exchange server contains an unspecif...
1081    connectwise screenconnect contains an authenti...
Name: shortDescription, Length: 1082, dtype: object

In [70]:
# Função de Tokenização
def tokenizacao (corpus):
    tokens = word.tokenize(corpus)
    return tokens 

In [21]:
# Função remover stop words
def remove_stop_words(corpus):
    # Carrega lista de stop words do inglês
    sw_en = set(stopwords.words('english'))
    tokens = corpus
    return corpus

In [23]:
# Função de Lematização
def lematizacao (df, nome_coluna):
    df[nome_coluna] = df[nome_coluna].map(lambda x: WordNetLemmatizer.lemmatize(x))
    return df

In [35]:
# Vetorizar coluna
vetorizar = CountVectorizer(max_features=100)
bow = vetorizar.fit_transform(catalogoVulnerabilidades.shortDescription)

print(bow.shape)

(1082, 100)


In [39]:
# Tranforma matriz esparsa em Dataframe
df_shortDescription_bow = pd.DataFrame.sparse.from_spmatrix(bow, columns=vetorizar.get_feature_names_out())

df_shortDescription_bow.head()

,access,adobe,affect,affected,after,allow,allows,an,and,apple,...,use,user,users,via,vulnerability,web,when,which,windows,with
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,1,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,1,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
4,0,0,0,0,0,1,0,1,1,0,...,0,1,0,0,1,0,0,0,0,0
